In [6]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

load_dotenv("../.env", override=True)

llm = ChatOpenAI(
    model="llama-3.3-70b-versatile",
    base_url="https://api.groq.com/openai/v1",
    openai_api_key=os.getenv("GROQ_API_KEY"),
    temperature=0
)

response = llm.invoke("Say hello in one short sentence.")
print(response.content)

Hello, it's nice to meet you.


In [8]:
def load_pdf(pdf_path: str):
    loader = PyPDFLoader(pdf_path)
    return loader.load()
docs = load_pdf("/Users/emmanuelreddy/Desktop/rag2/rag-grok/data/POST-SESSION RESOURCES - S4.pdf")
docs[0].page_content[:300]


"POST-SESSION  RESOURCES  \nSession  4:  The  AI  Prototyper's  Sandbox:  Mastering  Google  AI  Studio  \n  \nThis\n \nresource\n \npack\n \nprovides\n \nthe\n \ntools,\n \ntemplates,\n \nand\n \nreference\n \nmaterial\n \nto\n \nhelp\n \nyou\n \ndeepen\n \nyour\n \nunderstanding\n \nof\n \ntoday’s\n \nconcepts\n \nand\n \napply\n \nthem\n \ndir"

In [9]:

def chunk_documents(documents):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    return splitter.split_documents(documents)
chunks = chunk_documents(docs)
len(chunks)


11

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vector_db = FAISS.from_documents(chunks, embeddings)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable